In [1]:
import random # psuedo randomness generator
import numpy as np # import numerical python
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from cadCAD.configuration import Configuration

#imported some addition utilities to help with configuration set-up
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import append_configs
from cadCAD import configs

In [2]:
# What does this do?
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})


In [3]:
# Params

initial_conditions = {
    'village_folk': {i:{'stuff': np.random.randint(1,10), 'tokens':0} for i in range(100)}, # initialize village folk and their stuff and cred_castle balances
    'castle': 0, # the cred castle
    'castle_token_supply': 0, # the amount of castle tokens
}

In [4]:
# Everyone in CredLand is crazy 
def villagers_be_crazy(params, step, sL, s):
    n = len(s['village_folk']) #get the length of the list of villager indices
    #print('go crazy')
    index = np.random.randint(0, n) #pick a random integer from (0,.., n-1)
    p = np.random.randint(55, 60)
    if (p < 50):
        #print("if 1")
        action = 'buy_castle_token'
        #i need to buy FROM someone
        #select the set of village_folk who have tokens
        holders = [i for i in range(n) if s['village_folk'][i]['tokens']>0]
        if len(holders)>0:
            friend = np.random.choice(holders)
        else:
            action = None
            friend = None
        

    #you cannot sell tokens you don't have
    elif (p > 80)&(s['village_folk'][index]['tokens']>0):
        #print("if 2")
        action = 'sell_castle_token'
        #i need to sell TO someone
        #select the set of village_folk who have stuff
        stuff_havers = [i for i in range(n) if s['village_folk'][i]['stuff']>0]
        friend = np.random.choice(stuff_havers)
    
    elif (p > 50):
        #print("if 3")
        action = 'contribute'
        friend = None
    
    #print("Carry out %s for villager %s" % (action, index))
    #print(s['village_folk'][index])
    return ({'action': action, 'villager': index, 'friend':friend})

#    return (x, y)

In [5]:
# How villagers build the castle 
def build_castle(params, step, sL, s, _input):
    action = _input['action']
    #print(action)
    y = 'castle' #this function updates the castle state
    x = s[y]
    if action == 'contribute':
        x += 1
    #print(y)
    #print(x)
    return (y, x)

In [6]:
def account_castle_tokens(params, step, sL, s, _input):
    action = _input['action']
    #print(action)
    y = 'castle_token_supply' #this function updates the castle state
    x = s[y]
    if action == 'contribute':
        x += 1
    #print(y)
    #print(x)
    return (y, x)

In [7]:
# How villagers build the castle 
def royal_decree(params, step, sL, s, _input):
    action = _input['action']
    #print(action)
    y = 'village_folk' #this function updates the castle state
    x = s[y]
    #print(y)
    #print(type(x))
    #print(x)
    villager_index = _input['villager']
    if action == 'contribute':
        x[villager_index]['tokens'] = x[villager_index]['tokens'] +1
    #print(type(x))
    #print(x)
    
    return (y, x)

In [8]:
# How villagers sell castle tokens
def sell_castle_tokens(params, step, sL, s, _input):
    action = _input['action']
    #print('sell_castle_token')
    y = 'village_folk'
    x = s[y]
    
    if action == 'sell_castle_token':
        #villager_index = _input['person'] #you used the key "villager" not 'person' 
        villager_index = _input['villager']
        friend_index = _input['friend']
        #token accounting
        x[villager_index][1] -= 1 
        x[friend_index][1] += 1 #typo index <-- indes
        #stuff accounting
        x[villager_index][0] += 1 
        x[friend_index][0] -= 1 #typo index <-- indes

    return(y, x)

In [9]:
# How villagers buy castle tokens 
def buy_castle_tokens(params, step, sL, s, _input):
    action = _input['action']
    #print('buy_castle_token')
    y = 'village_folk'
    x = s[y]

    if action == 'buy_castle_token':
        #villager_index = _input['person'] #you used the key "villager" not 'person' 
        villager_index = _input['villager']
        friend_index = _input['friend']
        #token accounting
        x[villager_index][1] += 1 
        x[friend_index][1] -= 1 #typo index <-- indes
        #stuff accounting
        x[villager_index][0] -= 1 
        x[friend_index][0] += 1 #typo index <-- indes
    
    return(y, x)

#note if you are copy pasting code, it probably means you could collapse it into one function
#eg, buy and sell here are the same thing but with a negative number for sell

In [10]:
# Everyone in CredLand is crazy 
partial_state_update_blocks = [
    { 
        'policies': {
            'villagers_be_crazy': villagers_be_crazy
        },
        'variables': {
            'castle': build_castle,
            'village_folk': royal_decree,
            'castle_token_supply': account_castle_tokens
 #           'buy_castle_tokens': buy_castle_tokens,
 #           'sell_castle_tokens': sell_castle_tokens
        }
    }
]

In [11]:
len(partial_state_update_blocks)

1

In [12]:
simulation_parameters = {
    'T': range(50),
    'N': 1,
    'M': {}
}

In [13]:
# i replaced this with another cell which uses "append_configs" utility instead 
#---
# config = Configuration(initial_state=initial_conditions,
#                        partial_state_update_blocks=partial_state_update_blocks,
#                        sim_config=simulation_parameters
#                       )


In [14]:
append_configs(
    initial_state=initial_conditions, #dict containing variable names and initial values
    partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
    sim_configs=simulation_parameters #dict containing simulation parameters
)

In [15]:
# What does this do and why doesn't it work?
# this provides instructions to your computer on how to run your model
# since nothing actually gets executed until this cell, errors the model above
# trigger errors when you run this cell
from cadCAD.engine import ExecutionMode, ExecutionContext
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)

In [16]:
from cadCAD.engine import Executor
from cadCAD import configs
simulation = Executor(exec_context=local_mode_ctx, configs=configs)

In [17]:
import pandas as pd
raw_system_events, tensor_field, sessions = simulation.execute()

# Simulation Result Types:
# raw_system_events: List[dict] 
# tensor_field: pd.DataFrame

# Result System Events DataFrame
simulation_result = pd.DataFrame(raw_system_events)

Configurations Length: 1
Execution Method: local_simulations
Execution Mode: single_threaded


In [18]:
simulation_result

,village_folk,castle,castle_token_supply,simulation,run,substep,timestep
0,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",0,0,0,1,0,0
1,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",1,1,0,1,0,1
2,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",2,2,0,1,0,2
3,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",3,3,0,1,0,3
4,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",4,4,0,1,0,4
5,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",5,5,0,1,0,5
6,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",6,6,0,1,0,6
7,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",7,7,0,1,0,7
8,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",8,8,0,1,0,8
9,"{0: {'stuff': 9, 'tokens': 0}, 1: {'stuff': 8,...",9,9,0,1,0,9
